# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## Lesson

The saying goes, "Writing is rewriting." It turns out, **Claude can often improve the accuracy of its response when asked to do so**!

There are many ways to prompt Claude to "think again". The ways that feel natural to ask a human to double check their work will also generally work for Claude. (Check out our [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### Examples

In this example, we ask Claude to come up with ten words... but one or more of them isn't a real word.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

**Asking Claude to make its answer more accurate** fixes the error! 

Below, we've pulled down Claude's incorrect response from above and added another turn to the conversation asking Claude to fix its previous answer.

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

But is Claude revising its answer just because we told it to? What if we start off with a correct answer already? Will Claude lose its confidence? Here, we've placed a correct response in the place of `first_response` and asked it to double check again.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

You may notice that if you generate a respnse from the above block a few times, Claude leaves the words as is most of the time, but still occasionally changes the words even though they're all already correct. What can we do to mitigate this? Per Chapter 8, we can give Claude an out! Let's try this one more time.

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

请用上述代码多试几次生成回复，你会发现现在的克劳德更能坚持己见了。

你还可以通过提示链​​让克劳德优化自己的回答​​。下面我们让克劳德先写个故事，再让它改进自己写的故事。个人审美或有不同，但多数人都会觉得克劳德改写的第二版更出色。

先来看看克劳德写的故事初稿。

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Now let's have Claude improve on its first draft.

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

这种替换形式极其强大。我们一直在使用替换占位符来传入列表、词汇、Claude之前的回答等内容。您还可以​​通过替换实现所谓的"函数调用"——即要求Claude执行某项功能，然后将其结果再次交由Claude进行深度处理​​。其运作原理与其他替换操作如出一辙。下一附录将对此展开更详细的探讨。

以下是通过将Claude初次调用的结果嵌入到后续更复杂调用中的另一个范例。让我们从第一个提示开始（这次包含了Claude回答的预设内容）。

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

Let's pass this list of names into another prompt.

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

Now that you've learned about prompt chaining, head over to Appendix 10.2 to learn how to implement function calling using prompt chaining.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

In [ ]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

In [ ]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

In [ ]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))